In [1]:
#!ls
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10


In [ ]:
# concatenate
#!cat *.tsv> eval_data_test.tsv

#   
#!less eval_data_test.tsv
#
#!wc -l eval_data_test.tsv
#
#!wc -l *.tsv
#

## ici, c'est une zone markdown

In [ ]:
"ici une cellule d'exécution des scrpits" 

# open the file
with open ("/Savoirs_env/db_1_savoirs_XMLtei/BNU_02_Colin.xml", "r", encoding="utf_8") as file:
    # read the file
    text = file.read()
    # print it 
    print(text)

In [ ]:
# open the file
with open ("/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/db_2_NERbeyond/BNU_01_Didier_text_beyond.xml", "r", encoding="utf_8") as file:
    # read the file
    text = file.read()
    # print it 
    print(text)

In [ ]:
# open the file
with open ("/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/db_4_OutputNerBeyond/BNU_01_Didier_text_beyond.conll", "r", encoding="utf_8") as file:
    # read the file
    text = file.read()
    # print it 
    print(text)

In [ ]:
# operation system
import os
# regular expression
import re
# retrives file support
import glob
# inform the path

# replacement from NER & Beyond to IOB format    
_replacements = {
            'B-TOP_GR': 'B-LOC',
            'B-PERSNAME_NAME': 'B-PER',
            'I-PERSNAME_NAME': 'I-PER',
            'I-TOP_GR': 'I-LOC',
            }

def _do_replace(text):
    return _replacements.get(text.group(0))

def replace_tags(text, _re=re.compile('|'.join(re.escape(r) for r in _replacements))):
        return _re.sub(_do_replace, text)

for file_path in glob.iglob('/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/db_4_OutputNerBeyond/*.conll'):
    #transform path into a readable file
    f_name = (file_path)
    #print(f_name)
    
    # open the variable to be read and split into words
    with open(f_name, 'r', encoding='utf8') as f:
        # read and split into words to count word in file
        text = f.read()
        #print(text)
    
    result = (replace_tags(text))
    print(result)
"""
# directory out
    output_dir = "db_BIO/"
    # new files out with original's name plus _text and its new format .txt
    results_file = "%s%s_BIO.conll"%(output_dir, os.path.splitext(os.path.basename(f_name))[0])
    print(results_file)
    # save it as blabla_BIO.CONLL
    with open(results_file, 'w') as fpout: 
        fpout.write(str(result))
        #print to verify the result
        #print(first)
  """
  

In [ ]:
# operation system
import os
# regular expression
import re
# retrives file support
import glob
# make a table
import pandas as pd
# regular expression
import re

# inform the path
for file_path in glob.iglob('/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/db_4_OutputNerBeyond/*.conll'):
    #transform path into a readable file
    f_name = (file_path)
    
    # open the variable to be read and split into words
    with open(f_name, 'r', encoding='utf8') as f:
        # read and split into words to count word in file
        file_name = f.read()
        
        # read the files with panda
        df_ = pd.read_csv(f_name, sep="\t", names=["TOK"])
        table_ = pd.DataFrame(df_)
        
        # split the column TOK in two GOLD with the IOB format and TOK_ with the tokenized text
        table_[['TOKEN','IOB']] = table_.TOK.str.split(" ",expand=True,)

        # delete the column from TABLE_HAND previews columns
        del table_ ['TOK']
        del table_ ['IOB']
        
        # delete the index
        blankIndex=[''] * len(table_)
        table_.index=blankIndex
        
        # it allows to display entire table
        pd.set_option("display.max_rows", None, "display.max_columns", None )
        #print(table_)
        
        # directory out
        output_dir = "/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/db_10_tokenized/"
        # new files out with original's name plus _text and its new format .txt
        results_file = "%s%s_TOKENIZED.tsv"%(output_dir, os.path.splitext(os.path.basename(f_hand))[0])
        print(results_file)
        # write to files
        with open(results_file,'w') as write_csv:
            write_csv.write(table_hand.to_csv(sep='\t', index=False))
            print(write_csv)
        

In [ ]:
# operation system
import os
# regular expression
import re
# retrives file support
import glob
# make a table
import pandas as pd
# regular expression
import re

# SpaCy NLP
import spacy
from spacy.training import offsets_to_biluo_tags
from spacy.lang.fr import French
from spacy.tokens import Doc

try:
    nlp = spacy.load('fr_dep_news_trf')
except OSError:
    #print("Do you have the spacy module fr_core_news_sm installed? If not run 'python -m spacy download fr_core_news_sm'")
    
    def custom_tokenize(text):
        words = text.split('|')
        return Doc(nlp.vocab, words=words)
    nlp.tokenizer = custom_tokenize

    # inform the path
    for file_path in glob.iglob('tokenized_text/*.txt'): 
        #transform path into a readable file
        f_name = (file_path)
        #print(f_name)

        # open the variable to be read and split into words
        with open(f_name, 'r', encoding='utf8') as f:
            # read and split into words to count word in file
            #text = f.read()

            words = f.read().replace('\n', '|').replace('||','|').replace('||','|')
            words = words[:-1] #remove last pipe
            #print(words)

            # apply SpaCy
            doc = nlp(words)

            entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
            tags = offsets_to_biluo_tags(doc, entities)

            # directory out
            output_dir = "SpaCy_bert/"
            # new files out with original's name plus _text and its new format .txt
            results_file = "%s%s_SpaCy_bert.tsv"%(output_dir, os.path.splitext(os.path.basename(f_name))[0])
            print(results_file)


            # save it as blabla_text.tok
            count = 0
            with open(results_file, 'w', encoding='utf8') as fpout: 
                for tok in words.split("|"):
                    tag = str(tags[count])
                    fpout.write(str(tok)+"\t"+tag+"\n")
                    count = count + 1
                fpout.close()
            #print("fini")


In [ ]:
# operation system
import os
# regular expression
import re
# retrives file support
import glob
# make a table
import pandas as pd
# regular expression
import re

#### HAND TEXT ####

# inform the path
for file_path in glob.iglob('/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/hand_pandas/*txt'):
    #transform path into a readable file
    f_hand = (file_path)
    print(f_hand)
    
    # open the variable to be read and split into words
    with open(f_hand, 'r', encoding='utf8') as f:
        # read and split into words to count word in file
        t_hand = f.read()
        #print(t_hand)
        
        #### PANDAS ####
        
        # read the files with panda
        df_hand = pd.read_csv(f_hand, sep="\t", names=["TOK"])
        #print(f_hand)
        
        table_hand = pd.DataFrame(df_hand)
        #print(table_hand)
        
        # split the column TOK in two GOLD with the IOB format and TOK_ with the tokenized text
        table_hand[['TOKEN','IOB']] = table_hand.TOK.str.split(" ",expand=True,)
        #print(table_hand)
        
        # delete the column from TABLE_HAND previews columns
        del table_hand ['TOK']
        #print(table_hand)
        
        # delete the index
        blankIndex=[''] * len(table_hand)
        table_hand.index=blankIndex
        #print(table_hand)
        
        # it allows to display entire table
        pd.set_option("display.max_rows", None, "display.max_columns", None )
        print(table_hand)
        
        """   
        #output
        
        # directory out
        output_dir = "/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/db_10_tokenIOB/"
        # new files out with original's name plus _text and its new format .txt
        results_file = "%s%s_IOB_TOKEN.tsv"%(output_dir, os.path.splitext(os.path.basename(f_hand))[0])
        print(results_file)
        # write to files
        with open(results_file,'w') as write_csv:
            write_csv.write(table_hand.to_csv(sep='\t', index=False))
            print(write_csv)
            
            """
        


In [ ]:
!pwd

In [ ]:
# open the file
with open ("/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/db_8_SpaCy_results/SpaCy_prediction/prediction_lg/BNU_01_Didier_text_SpaCy_lg_prediction.tsv", "r", encoding="utf_8") as file:
    # read the file
    text = file.read()
    # print it 
    print(text)

In [ ]:
!pwd
!ls


In [13]:
# open the file
with open ("/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/db_8_SpaCy_results/SpaCy_precision_rappel_LOC_PER/lg_PER_average.tsv", "r", encoding="utf_8") as file:
    # read the file
    text = file.read()
    # print it 
    print(text)

"Measure	ent_type	partial	strict	exact"
"correct	143	135	135	135"
"incorrect	0	0	8	8"
"partial	0	8	0	0"
"missed	60	60	60	60"
"spurious	88	88	88	88"
"possible	203	203	203	203"
"actual	231	231	231	231"
"precision	0.619	0.602	0.584	0.584"
"recall	0.704	0.685	0.665	0.665"
"f1	0.659	0.641	0.622	0.622"



In [ ]:
# open the file
with open ("/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/comparaison.txt", "r", encoding="utf_8") as file:
    # read the file
    text = file.read()
    # print it 
    print(text)

In [ ]:
# operation system
import os
# regular expression
import re
# retrives file support
import glob
# make a table
import pandas as pd
# regular expression
import re


# inform the path
for file_path in glob.iglob('/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/db_8_SpaCy_results/SpaCy_precision_rappel_LOC_PER/*.tsv'):
    #transform path into a readable file
    f_name = (file_path)
    print(f_name)
    
    # open the variable to be read and split into words
    with open(f_name, 'r', encoding='utf8') as f:
        # read and split into words to count word in file
        t_print = f.read()
        print(t_print)


In [ ]:
!pwd

In [ ]:
"""Visualize the data with Streamlit and spaCy."""
import streamlit as st
from spacy import displacy
import srsly
import typer

file_paths = "/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/savoirs_NER/training/model-best/meta.json"


#@st.cache(allow_output_mutation=True)
"""
def load_data(filepath):
    examples = list(srsly.read_json(filepath))
    rows = []
    n_total_ents = 0
    n_no_ents = 0
    labels = set()
    for eg in examples:
        row = {"text": eg["text"], "ents": eg.get("spans", [])}
        n_total_ents += len(row["ents"])
        if not row["ents"]:
            n_no_ents += 1
        labels.update([span["label"] for span in row["ents"]])
        rows.append(row)
    return rows, labels, n_total_ents, n_no_ents
    """

def main(file_paths: str):
    files = [p.strip() for p in file_paths.split(",")]
    st.sidebar.title("Data visualizer")
    st.sidebar.markdown(
        "Visualize the annotations using [displaCy](https://spacy.io/usage/visualizers) "
        "and view stats about the datasets."
    )
    data_file = st.sidebar.selectbox("Dataset", files)
    data, labels, n_total_ents, n_no_ents = load_data(data_file)
    displacy_settings = {
        "style": "ent",
        "manual": True,
        "options": {"colors": {label: "#d1bcff" for label in labels}},
    }
    st.header(f"{data_file} ({len(data)})")
    wrapper = "<div style='border-bottom: 1px solid #ccc; padding: 20px 0'>{}</div>"
    for row in data:
        html = displacy.render(row, **displacy_settings).replace("\n\n", "\n")
        st.markdown(wrapper.format(html), unsafe_allow_html=True)

    st.sidebar.markdown(
        f"""
    | `{data_file}` | |
    | --- | ---: |
    | Total examples | {len(data):,} |
    | Total entities | {n_total_ents:,} |
    | Examples with no entities | {n_no_ents:,} |
    """
    )


if __name__ == "__main__":
    try:
        typer.run(main)
    except SystemExit:
        pass


In [ ]:
# operation system
import os
# retrives file support
import glob

# input directory
input_dir = "/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/db_10_tokenIOB/"    

### ALL outputs
# output directory train
output_dir_train = "/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/13_db_spaCY_trainning/train_data/"
# output directory val
output_dir_val = "/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/13_db_spaCY_trainning/eval_data/"
# output directory test
output_dir_test = "/Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/13_db_spaCY_trainning/test_data/"

# open all tsv files in this directory 
for file_name in glob.glob(input_dir + "*.tsv"):
    #print(file_name)
    with open(file_name, "r", encoding="UTF-8") as fp:
        print_f = fp.read()
        #print(print_f)
        
        # counting the number of lines
        num_lines = len(print_f.split("\n"))
        
        # stablishing the metrics of divison for each file 
        train_r, val_r, test_r = num_lines*0.7, num_lines*0.2, num_lines*0.1
        
        # salving the results with their original's name plus its new categories
        f_train = open("%s%s_train.tsv"%(output_dir_train, os.path.splitext(os.path.basename(file_name))[0]), "w", encoding="UTF-8")
        f_val = open("%s%s_val.tsv"%(output_dir_val, os.path.splitext(os.path.basename(file_name))[0]), "w", encoding="UTF-8")
        f_test = open("%s%s_test.tsv"%(output_dir_test, os.path.splitext(os.path.basename(file_name))[0]), "w", encoding="UTF-8")   
        
        count = 0
        # parsing each line
        lines2 = print_f.split("\n")
        # loop for division
        for l2 in lines2:
            #print(count)
            if count < train_r:
                f_train.write(l2+"\n")
                #print("in train: "+l2+"\n")
            elif count >= train_r and count < train_r + val_r:
                f_val.write(l2+"\n")
                #print("in val: "+l2+"\n")
            else:
                f_test.write(l2+"\n")
                #print("in test: "+l2+"\n")
            count = count + 1
            #print(count)
        
        # closing the variable result
        f_train.close()
        f_val.close()
        f_test.close()
               

In [ ]:
for d in *; do echo $d ; python -m spacy convert $d /Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/13_db_spaCY_trainning/train_data/ /Users/alexsoares/Desktop/EHESS/dev/Savoirs_env/14_db_json_spaCy/train_json/ -t json -s -c iob ; done


In [ ]:
!python -m spacy convert /input_file.tsv /output_file -t spacy

In [ ]:
!python3 -m spacy train config.cfg --output ./training 
                            --paths.train ./corpus/train.spacy 
                            --paths.dev ./corpus/dev.spacy

In [ ]:
#!/bin/bash

#File: tree-md

tree=$(tree -tf --noreport -I '*~' --charset ascii $1 |
       sed -e 's/| \+/  /g' -e 's/[|`]-\+/ */g' -e 's:\(* \)\(\(.*/\)\([^/]\+\)\):\1[\4](\2):g')

printf "# Project tree\n\n${tree}"



In [ ]:
"https://isolution.pro/fr"

In [ ]:
"https://isolution.pro/fr/t/pytorch/pytorch-universal-workflow-of-machine-learning/flux-de-travail-universel-de-l-apprentissage-automatique"

In [ ]:

! pwd


In [ ]:
!python3 -m spacy project run all


In [ ]:
"https://ichi.pro/fr/comment-le-traitement-du-langage-naturel-pnl-changera-t-il-le-monde-143158135786453"

In [ ]:
https://algorithmia.com/blog/introduction-to-loss-functions
    https://www.youtube.com/watch?v=IVVVjBSk9N0
        https://github.com/llSourcell/loss_functions_explained
            https://rohanvarma.me/Loss-Functions/